In [1]:
import os
import sys
import numpy as np
from PIL import Image
from skimage import io, img_as_float
from scipy import signal
from skimage.color import rgb2gray
from skimage.transform import resize
import math
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import torchvision

In [3]:
# To ignore warnings when splitting images
import warnings
warnings.simplefilter('ignore')

### Defining model architecture

In [4]:
class ConvBlock(nn.Module):
    def __init__(self, m, in_filters, out_filters, kernel_size=3, batchnorm=True, last_activ=F.relu):
        super(ConvBlock, self).__init__()
        if m==50:
            self.bn = batchnorm
            self.last_activ = last_activ
            self.c1 = nn.Conv2d(in_filters, out_filters, kernel_size, padding=1)
            self.b1 = nn.BatchNorm2d(out_filters)
            self.c2 = nn.Conv2d(out_filters, out_filters, kernel_size, padding=1)
            self.b2 = nn.BatchNorm2d(out_filters)
        else:
            self.bn = batchnorm
            self.last_activ = last_activ
            self.encoder = torchvision.models.vgg16(pretrained=True).features
            #self.c1 = nn.Conv2d(in_filters, out_filters, kernel_size, padding=1)
            self.c1 = self.encoder[m]
            self.b1 = nn.BatchNorm2d(out_filters)
            #self.c2 = nn.Conv2d(out_filters, out_filters, kernel_size, padding=1)
            self.c2 = self.encoder[m+2]
            self.b2 = nn.BatchNorm2d(out_filters)
    def forward(self, x):
        x = self.c1(x)
        if self.bn: x = self.b1(x)
        x = F.relu(x)
        x = self.c2(x)
        if self.bn: x = self.b2(x)
        x = self.last_activ(x)
        return x

In [5]:
class ContractiveBlock(nn.Module):
    def __init__(self, m, in_filters, out_filters, conv_kern=3, pool_kern=2, dropout=0.5, batchnorm=True):
        super(ContractiveBlock, self).__init__()
        self.c1 = ConvBlock(m=m, in_filters=in_filters, out_filters=out_filters, kernel_size=conv_kern, batchnorm=batchnorm)
        self.p1 = nn.MaxPool2d(kernel_size=pool_kern, ceil_mode=True)
        self.d1 = nn.Dropout2d(dropout)
    def forward(self, x):
        c = self.c1(x)
        return c, self.d1(self.p1(c))

In [6]:
class ExpansiveBlock(nn.Module):
    def __init__(self,in_filters1, in_filters2, out_filters, tr_kern=3, conv_kern=3, stride=2, dropout=0.5):
        super(ExpansiveBlock, self).__init__()
        self.t1 = nn.ConvTranspose2d(in_filters1, out_filters, tr_kern, stride=2, padding=1, output_padding=1)
        self.d1 = nn.Dropout(dropout)
        self.c1 = ConvBlock(50,out_filters+in_filters2, out_filters, conv_kern)
        
    def forward(self, x, contractive_x):
        x_ups = self.t1(x)
        x_concat = torch.cat([x_ups, contractive_x], 1)
        x_fin = self.c1(self.d1(x_concat))
        return x_fin

In [7]:
class Unet(nn.Module):
    def __init__(self, n_labels, n_filters=64, p_dropout=0.5, batchnorm=True):
        super(Unet, self).__init__()
        filter_dict = {}
        filt_pair = [3, n_filters]
        m=[0,5,10,17,24]
        for i in range(4):
            self.add_module('contractive_'+str(i), ContractiveBlock(m[i], filt_pair[0], filt_pair[1], batchnorm=batchnorm))
            filter_dict['contractive_'+str(i)] = (filt_pair[0], filt_pair[1])
            filt_pair[0] = filt_pair[1]
            filt_pair[1] = filt_pair[1] * 2
        
        self.bottleneck = ConvBlock(50,filt_pair[0], filt_pair[1], batchnorm=batchnorm)
        filter_dict['bottleneck'] = (filt_pair[0], filt_pair[1])
        
        for i in reversed(range(4)):
            self.add_module('expansive_'+str(i), ExpansiveBlock(filt_pair[1], filter_dict['contractive_'+str(i)][1], filt_pair[0]))
            filter_dict['expansive_'+str(i)] = (filt_pair[1], filt_pair[0])
            filt_pair[1] = filt_pair[0]
            filt_pair[0] = filt_pair[0] // 2
        
        self.output = nn.Conv2d(filt_pair[1], n_labels, kernel_size=1)
        filter_dict['output'] = (filt_pair[1], n_labels)
        self.filter_dict = filter_dict
    
    def forward(self, x):
        c00, c0 = self.contractive_0(x)
        c11, c1 = self.contractive_1(c0)
        c22, c2 = self.contractive_2(c1)
        c33, c3 = self.contractive_3(c2)
        bottle = self.bottleneck(c3)
        u3 = F.relu(self.expansive_3(bottle, c33))
        u2 = F.relu(self.expansive_2(u3, c22))
        u1 = F.relu(self.expansive_1(u2, c11))
        u0 = F.relu(self.expansive_0(u1, c00))
        return F.softmax(self.output(u0), dim=1)
        

In [8]:
net = Unet(2).cuda()

In [9]:
net.load_state_dict(torch.load('output_models/torch/unet_vgg16_inunet2.pth'))
print(net)

Unet(
  (contractive_0): ContractiveBlock(
    (c1): ConvBlock(
      (encoder): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace)
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU(inplace)
        (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (6): ReLU(inplace)
        (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (8): ReLU(inplace)
        (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (11): ReLU(inplace)
        (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (13): ReLU(inplace)
        (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   

In [10]:
# splits the image into 256*256 patches and returns a 2d list of patches
def create_patches(im):
    print(im.dtype)
    im = Image.fromarray(im)
    print(im.size)
    new_size = 256*int(im.size[0]/256),256*int(im.size[1]/256)
    print(new_size)
    im = im.resize(new_size)
    im = np.array(im)
    image_patches = []
    for i,x in enumerate(range(0,new_size[1],256)):
        row_patches= []
        for j,y in enumerate(range(0,new_size[0],256)):
            new_im = im[x:x+256,y:y+256]
            row_patches.append(new_im)
        image_patches.append(row_patches)
    return image_patches

In [11]:
# Uncomment the line with Normalize if working with Normalized images during training
data_tranform = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
#                   torchvision.transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
                    ])

In [12]:
# makes the output clearer and less fuzzy by whitening the pixels which have value above threshold
def digitize(x,threshold=0.3):
    if x>threshold:
        return min(0.99,2.0*x)
    else:
        return 0.
digitize = np.vectorize(digitize)

In [13]:
# label_path is the path of manually segmented vessel map (not used currently but can be used for comparison)
# target_mask is the path where you wish to save the vessel map
def evaluate(image_path):
    orig_im = io.imread('DRIVE4/test/input/1.png')
    #orig_im=orig_im[:,:,0:3]
    orig_im=resize(orig_im,(512,512,3))
    patched_image = create_patches(orig_im)
    full_image = []
    for i, row in enumerate(patched_image):
        row_im = []
        for j, im in enumerate(row):
            img = data_tranform(im).unsqueeze(0).cuda()
            pred = net(img).cpu().detach().numpy()[0][0]
            row_im.append(pred)
        full_image.append(row_im)
    stitch = []
    for i in full_image:
        col_im = []
        for j in i:
            col_im.append(j)
        col_im = np.hstack(col_im)
        stitch.append(col_im)
    stitch = np.vstack(stitch)
    stitch = digitize(stitch)
    orig_im = io.imread('DRIVE4/test/input/1.png')
    #orig_im=orig_im[:,:,0:3]
    orig_im=resize(orig_im,(512,512,3))
    stitch = resize(stitch,(orig_im.shape[0], orig_im.shape[1]),anti_aliasing=True)
    #io.imsave(stitch)
    f = plt.figure(figsize=(15,15))
    f.add_subplot(1,2,1)
    plt.imshow(orig_im)
    f.add_subplot(1,2,2)
    plt.imshow(stitch,cmap='gray')

In [14]:
torch.cuda.empty_cache()

In [15]:
def generate_vessel_map(orig_im):
    patched_image = create_patches(orig_im)
    
    full_image = []
    for i, row in enumerate(patched_image):
        row_im = []
        for j, im in enumerate(row):
            img = data_tranform(im).unsqueeze(0).cuda()
            pred = net(img).cpu().detach().numpy()[0][0]
            row_im.append(pred)
        full_image.append(row_im)
    stitch = []
    for i in full_image:
        col_im = []
        for j in i:
            col_im.append(j)
        col_im = np.hstack(col_im)
        stitch.append(col_im)
    stitch = np.vstack(stitch)
    stitch = digitize(stitch)
    
    stitch = resize(stitch,(orig_im.shape[0], orig_im.shape[1]),anti_aliasing=True)
    return stitch

In [16]:
def choose_side(box_blur_orig, channel_orig, mask_orig):
    box_blur = box_blur_orig.flatten()
    channel = channel_orig.flatten()
    mask = mask_orig.flatten()
    
    # selecting all pixels with mask value greater than 0.1
    vessel = (mask>0.1)
    
    retval = np.zeros(box_blur.shape)
    # assign local average to selected pixels
    retval[vessel] = box_blur[vessel]
    # keep remaining ones same as original
    retval[~vessel] = channel[~vessel]
    return retval.reshape(channel_orig.shape)

In [17]:
# removes vessel from each channel
def remove_vessel(channel,mask):
    # coarse adjustments
    # local averaging over 32*32 patch
    box_blur = signal.convolve2d(channel,np.ones((32,32)),boundary='wrap',mode='same')
    box_blur = box_blur/1024
    cured_channel = choose_side(box_blur, channel, mask)   
    channel = cured_channel

    # local averaging over 16*16 patch
    box_blur = signal.convolve2d(channel,np.ones((16,16)),boundary='wrap',mode='same')
    box_blur = box_blur/256
    cured_channel = np.zeros(channel.shape)
    cured_channel = choose_side(box_blur, channel, mask)
    channel = cured_channel
    
    # finer adjustments
    # local averaging over 8*8 patch
    box_blur = signal.convolve2d(channel,np.ones((8,8)),boundary='wrap',mode='same')
    box_blur = box_blur/64
    cured_channel = np.zeros(channel.shape)
    cured_channel = choose_side(box_blur, channel, mask)
    channel = cured_channel
    
    # local averaging over 4*4 patch
    box_blur = signal.convolve2d(channel,np.ones((4,4)),boundary='wrap',mode='same')
    box_blur = box_blur/16
    cured_channel = np.zeros(channel.shape)
    cured_channel = choose_side(box_blur, channel, mask)
    return cured_channel

In [18]:
# given an image and mask, calls remove_vessel on each channel of image
def cure_image(orig_im, mask):
    r,g,b = remove_vessel(orig_im[:,:,0],mask),remove_vessel(orig_im[:,:,1],mask),remove_vessel(orig_im[:,:,2],mask)
    cured_image =  np.dstack((r, g, b))
    return cured_image

In [19]:
def generate_output(image_path, target_dir):
    orig_im = (io.imread(image_path))
    mask = img_as_float(rgb2gray(generate_vessel_map(orig_im)))
    
    # new addition to remove stitching issues
    # move the image along both axes by 10 pixels 
    # and then add back the mask to orginal mask by shifting it back by 10 pixels
    mask_flip = img_as_float(rgb2gray(generate_vessel_map(np.roll(orig_im,shift=10,axis=(0,1)))))
    mask = np.clip(a=mask+np.roll(mask_flip,shift=-10,axis=(0,1)),a_min=0., a_max=0.99)
    cured_output = cure_image(img_as_float(orig_im),mask)

    image_name = image_path[image_path.rfind('/')+1:]
    image_name = image_name[:image_name.find('.')]
    #target_dir = target_dir + image_name + "/"
    os.system("mkdir -p "+ target_dir)
    print (target_dir)
    #io.imsave(target_dir + image_name + "_orig.jpg", orig_im)
    io.imsave(fname=target_dir + image_name+".png",arr=mask)
    #io.imsave(target_dir + image_name+"_clean.jpg",cured_output)
    return mask,cured_output

In [20]:
for i in range(1,21):
    image_path = "DRIVE4/test/input/%d.png"%i
    mask, out = generate_output(image_path, 'data/test_outputs/vgg16/')

    #f = plt.figure(figsize=(20,30))
    #a = f.add_subplot(1,3, 1)
    #im = io.imread(image_path)
    #plt.imshow(im)
    #f.add_subplot(1,3, 2)
    #plt.imshow(mask,cmap='gray')
    #f.add_subplot(1,3, 3)
    #plt.imshow(out)

uint8
(512, 512)
(512, 512)
uint8
(512, 512)
(512, 512)
data/test_outputs/vgg16/
uint8
(512, 512)
(512, 512)
uint8
(512, 512)
(512, 512)
data/test_outputs/vgg16/
uint8
(512, 512)
(512, 512)
uint8
(512, 512)
(512, 512)
data/test_outputs/vgg16/
uint8
(512, 512)
(512, 512)
uint8
(512, 512)
(512, 512)
data/test_outputs/vgg16/
uint8
(512, 512)
(512, 512)
uint8
(512, 512)
(512, 512)
data/test_outputs/vgg16/
uint8
(512, 512)
(512, 512)
uint8
(512, 512)
(512, 512)
data/test_outputs/vgg16/
uint8
(512, 512)
(512, 512)
uint8
(512, 512)
(512, 512)
data/test_outputs/vgg16/
uint8
(512, 512)
(512, 512)
uint8
(512, 512)
(512, 512)
data/test_outputs/vgg16/
uint8
(512, 512)
(512, 512)
uint8
(512, 512)
(512, 512)
data/test_outputs/vgg16/
uint8
(512, 512)
(512, 512)
uint8
(512, 512)
(512, 512)
data/test_outputs/vgg16/
uint8
(512, 512)
(512, 512)
uint8
(512, 512)
(512, 512)
data/test_outputs/vgg16/
uint8
(512, 512)
(512, 512)
uint8
(512, 512)
(512, 512)
data/test_outputs/vgg16/
uint8
(512, 512)
(512, 512)
